# Alignment and centrality

#### In this notebook, we estimate alignment in conversations and quantify to which amount alignment  is influenced by the centrality of the interlocutors.
---

In [1]:
import pickle
import dill

import numpy as np
import pandas as pd
import pymc3 as pm 
import seaborn as sns

from collections import defaultdict, Counter

import utils
from talkpages import WikiCorpusReader, WikiCorpus
from alignment import Alignment

%matplotlib inline
import matplotlib.pyplot as plt

> The conversations are taken from a selection of 10 topics from the Controversial TalkPages corpus.

In [2]:
TOPICS = ['religion',
          'science', 
          'politics', 
          'history', 
          'people',
          'philosophy', 
          'sports',
          'linguistics', 
          'psychiatry',
          'environment']

> To count alignment, we use a selection of marker categories and tokens from the LIWC dictionaries. There is no overlap between any two categories due to some preprocessing (`marker selection.ipynb`).

In [3]:
META_CATEGORIES = {'stylistic': [
                        'articles',
                        'negations',
                        'prepositions',
                        'numbers',
                        'pronouns'
                    ], 
                    'rhetoric': [
                        'tentative',   
                        'certainty',
                        'discrepancy',
                        'inclusive',
                        'exclusive'
                    ],
                    'discursive': [
                        'causation',
                        'insight',
                        'inhibition',
                        'communication',
                        'cognitive process',
                        'sensory process',
                        'motion'
                    ],
                    'stance': [
                        'optimism',
                        'anger',
                        'anxiety',
                        'sadness',
                    ]}


# Keep a list of category names for convenience.
CATEGORY_LIST = []
for cats in META_CATEGORIES.values():
    CATEGORY_LIST.extend(cats)
    
# Load the filtered lists of markers. 
with open('../../data/liwc/final.dict', 'rb') as f:
    MARKER_DICT = pickle.load(f)
    
    marker_list = []
    for markers in MARKER_DICT.values():
        marker_list.extend(markers)
    MARKER_LIST = list(set(marker_list))

> Last but not least, we need to choose a centrality measure: _eigenvector_ or _betweenness centrality_?

In [4]:
CENTRALITY = 'eigenvector'
# CENTRALITY = 'betweenness'

MODE = 'category'
# MODE = 'cnw'

MAX_ITERS = 100000
N_SAMPLES = 4000
TRACE_SIZE = 1000

CAUCHY_ALPHA = -2
CAUCHY_BETA = 3

----

> We focus on `category-not-word` alignment to exclude cases of lexical repetition ([Doyle & Frank 2016](http://www.aclweb.org/anthology/P16-1050), pp. 531-532).

In [ ]:
for TOPIC in TOPICS:
    
    print('{}\n{}'.format(TOPIC, '*'*15))
    
    # Load category-not-word alignment counts (Doyle & Frank, 2016)
    with open('./counts-{}/{}.dill'.format(MODE, TOPIC), 'rb') as f:
        if CENTRALITY == 'eigenvector':
            N_base_all, N_align_all, C_base_all, C_align_all, dyad2centrality, _, _, _ = dill.load(f)
        elif CENTRALITY == 'betweenness':
            N_base_all, N_align_all, C_base_all, C_align_all, _, dyad2centrality, _, _ = dill.load(f)
    
    # Statistical modelling
    for c, category in enumerate(CATEGORY_LIST):
        
        print('{}, {}\n{}'.format(TOPIC, category, '*'*20))
        
        # Data
        N_base, N_align, C_base, C_align = [], [], [], []
        A_centrality, B_centrality = [], []
        
        # collect the counts for this category of markers
        for dyad in N_base_all:
            
            if C_base_all[dyad][c] > N_base_all[dyad][c]:
                continue
            if C_align_all[dyad][c] > N_align_all[dyad][c]:
                continue
                
            try:
                A_centr, B_centr = dyad2centrality[dyad]
            except KeyError:
                continue
            N_base.append(N_base_all[dyad][c])
            C_base.append(C_base_all[dyad][c])
            N_align.append(N_align_all[dyad][c])
            C_align.append(C_align_all[dyad][c])
            A_centrality.append(A_centr)
            B_centrality.append(B_centr)
            
        if not any(N_base):
            print('N_base: all zeros.')
        if not any(N_align):
            print('N_align: all zeros.')
        if not any(C_align):
            print('C_align: all zeros.')
        if not any(C_base):
            print('C_base: all zeros.')
            
        if not (any(N_base) or any(N_align) or any(C_align) or any(C_base)):
            continue
        
        A_centrality =  utils.standardise(A_centrality)
        B_centrality =  utils.standardise(B_centrality)
        
        # A simple logistic model.
        with pm.Model() as model:
            # Parameters
            beta0 = pm.Cauchy('baseline intercept', alpha=CAUCHY_ALPHA, beta=CAUCHY_BETA)
            alpha0 = pm.Normal('alignment intercept', mu=0, sd=0.25)
            alpha1 = pm.Normal('coefficient centr(A)', mu=0, sd=1)
            alpha2 = pm.Normal('coefficient centr(B)', mu=0, sd=1)
            
            # Include a guessing coefficient for robust logistic regression
            # (cfr. J. Kruschke, 2014, 'Doing Bayesian data analysis', pp. 635-636)
            guess = pm.Beta('guessing coefficient', alpha=1, beta=9)  
            
            # Transformed parameters
            mu_base  = guess * 0.5 + (1-guess) * pm.math.invlogit(beta0)            
            mu_align = guess * 0.5 + (1-guess) * pm.math.invlogit(beta0+alpha0 + alpha1*A_centrality + alpha2*B_centrality)
            
            # Model
            base_count  = pm.Binomial('C_base' , p=mu_base , observed=C_base, n=N_base)
            align_count = pm.Binomial('C_align', p=mu_align, observed=C_align, n=N_align)
        

        # Inference
        with model:
            print(model.check_test_point())
    
            approx = pm.fit(n=MAX_ITERS, method='advi',
                            callbacks=[pm.callbacks.CheckParametersConvergence(diff='absolute')])

            print('Sampling {} ...'.format(N_SAMPLES), end=' ')
            full_trace = approx.sample(draws=N_SAMPLES)
            print('Done.')
            
            trace = full_trace[-TRACE_SIZE:]
            trace_df = pm.trace_to_dataframe(trace)
            trace_df.to_csv('./traces/{}/{}/{}-{}.csv'.format(MODE, CENTRALITY, TOPIC, category))
        
            print(pm.summary(trace))

            pm.traceplot(trace, varnames=['baseline intercept',
                                          'alignment intercept', 
                                          'coefficient centr(A)',
                                          'coefficient centr(B)',
                                          'guessing coefficient'])
            
            plt.savefig('plots/traceplots/{}/{}/{}-{}.pdf'.format(MODE, CENTRALITY, TOPIC, category))


            pm.plot_posterior(trace)
            plt.savefig('plots/posteriors/{}/{}/{}-{}.pdf'.format(MODE, CENTRALITY, TOPIC, category))


religion
***************
religion, articles
********************
baseline intercept                    -2.06
alignment intercept                    0.47
coefficient centr(A)                  -0.92
coefficient centr(B)                  -0.92
guessing coefficient_logodds__        -1.05
C_base                             -3596.19
C_align                          -494324.09
Name: Log-probability of test_point, dtype: float64


Average Loss = 85,104: 100%|██████████| 100000/100000 [08:17<00:00, 201.11it/s]   
Finished [100%]: Average Loss = 85,104
INFO:pymc3.variational.inference:Finished [100%]: Average Loss = 85,104


Sampling 4000 ... Done.
                          mean        sd  mc_error   hpd_2.5  hpd_97.5
baseline intercept   -1.540213  0.002321  0.000074 -1.544901 -1.535927
alignment intercept   0.105144  0.002264  0.000071  0.100988  0.109939
coefficient centr(A) -0.020343  0.001859  0.000058 -0.023980 -0.016577
coefficient centr(B) -0.003997  0.001983  0.000067 -0.008000 -0.000313
guessing coefficient  0.412109  0.000710  0.000021  0.410679  0.413399


/Users/mario/GitHub/virtualenvs/align/lib/python3.7/site-packages/matplotlib/axes/_base.py:3604: MatplotlibDeprecationWarning: 
The `ymin` argument was deprecated in Matplotlib 3.0 and will be removed in 3.2. Use `bottom` instead.
  alternative='`bottom`', obj_type='argument')


religion, negations
********************
baseline intercept                     -2.06
alignment intercept                     0.47
coefficient centr(A)                   -0.92
coefficient centr(B)                   -0.92
guessing coefficient_logodds__         -1.05
C_base                           -4168890.04
C_align                            -42492.48
Name: Log-probability of test_point, dtype: float64


Average Loss = 11,924:  46%|████▌     | 45941/100000 [03:45<06:44, 133.70it/s]    